In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix

from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Bidirectional
from itertools import product
from tqdm import tqdm


from Libs.load_data import DataLoader
from Libs.threshold import get_labels_physic
from Libs.keras_f1score import f1_m

2023-03-07 15:14:30.664263: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-07 15:14:33.090953: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-07 15:14:33.091058: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-03-07 15:14:39.213501: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [2]:
# initialize data loader
data_loader = DataLoader()
# get the grid
grid_X = data_loader.get_grid()
# get params dictionary
params = data_loader.get_params()
# get physic labels
grid_y = get_labels_physic(grid_X, params, alpha=2)

params

{'run': 30,
 'sigma': [0.3, 0.4, 0.5, 0.6],
 'theta': [0.01, 0.1, 0.5, 3],
 'mu': [0.8, 0.9, 1, 1.1],
 'delta': [0.01, 0.05, 0.1, 0.2, 0.25, 0.3, 0.4, 0.5, 0.6, 0.7],
 'N': 1000}

# LSTM model with multiple all theta parameters

Let's start seeing what is going to happen with training and testing the NN with all the configurations of theta parameters

In [3]:
run = 0
test_size = 0.33
random_state = 42

sigma_list = [0.5]
mu_list    = [1]
delta_list = [0.2]
theta_train_list = [0.01, 0.1, 0.5, 3]
theta_test_list  = [0.01, 0.1, 0.5, 3]

# select best params
idx = data_loader.get_standard_indexes()

In [13]:
def get_dataset_split(grid_x, grid_y, idx, test_size = 0.33, window_size = 10, overlap_size = 0):
    ###
    def build_df(X_configuration, y_configuration, window_size=window_size, overlap_size=overlap_size, label_treshold = 1):
        stride = window_size - overlap_size
        num_windows = (X_configuration.shape[-1]-window_size)//stride + 1

        windows = np.zeros((X_configuration.shape[0]*(num_windows-1),window_size))
        windows_label = np.zeros((y_configuration.shape[0]*(num_windows-1),window_size), dtype='bool')


        for i in range(X_configuration.shape[0]):
            tmp_windows = np.array([X_configuration[i,j:j+window_size] for j in range(0,stride*num_windows,stride)])
            tmp_windows_labels = np.array([y_configuration[i,j:j+window_size] for j in range(0,stride*num_windows,stride)])
            windows[i*(num_windows-1):(i+1)*(num_windows-1)] = tmp_windows[:-1,:]
            windows_label[i*(num_windows-1):(i+1)*(num_windows-1)] = tmp_windows_labels[1:,:]

        windows_label = np.sum(windows_label, axis=-1)
        windows_label[windows_label<label_treshold] = 0
        windows_label[windows_label>=label_treshold] = 1

        df = pd.DataFrame(windows, columns=[f't_{i}' for i in range(windows.shape[-1])]).sample(frac=1)
        y_df = pd.DataFrame({'future_flare':windows_label})
        df = pd.concat([df, y_df], axis=1)

        return df
    ###
    run_test_index = int((1-test_size) * params['run'])

    # build the dataframe
    X_configuration = []
    y_configuration = []
    # for s, t, d, m in tqdm(product(params['sigma'], params['theta'], params['delta'], params['mu'])):
    for s, t, d, m in tqdm(product(sigma_list, theta_train_list, delta_list, mu_list)):
        ti = params['theta'].index(t)
        mi = params['mu'].index(m)
        si = params['sigma'].index(s)
        di = params['delta'].index(d)
        X_configuration.append(grid_X[:run_test_index, ti, mi, si, di, :])
        y_configuration.append(grid_y[:run_test_index, ti, mi, si, di, :])

    X_configuration = np.hstack(X_configuration)
    y_configuration = np.hstack(y_configuration)
    # df training
    df_train = build_df(X_configuration, y_configuration)

    # build the dataframe
    X_configuration = []
    y_configuration = []
    # for s, t, d, m in tqdm(product(params['sigma'], params['theta'], params['delta'], params['mu'])):
    for s, t, d, m in tqdm(product(sigma_list, theta_test_list, delta_list, mu_list)):
        ti = params['theta'].index(t)
        mi = params['mu'].index(m)
        si = params['sigma'].index(s)
        di = params['delta'].index(d)
        X_configuration.append(grid_X[run_test_index:, ti, mi, si, di, :])
        y_configuration.append(grid_y[run_test_index:, ti, mi, si, di, :])
    X_configuration = np.hstack(X_configuration)
    y_configuration = np.hstack(y_configuration)
    # df test
    df_test = build_df(X_configuration, y_configuration)

    return df_train, df_test

df_train, df_test = get_dataset_split(grid_X, grid_y, idx, window_size=20, overlap_size=18)

2it [00:00, 13066.37it/s]
2it [00:00, 10824.01it/s]


This cell shows the amounts of class 0 and 1 for each data set

In [5]:
# number of classes
print('Training set:')
print(df_train['future_flare'].value_counts(), '\n')
print('Test set:')
print(df_test['future_flare'].value_counts(), '\n')

Training set:
0    29719
1    10081
Name: future_flare, dtype: int64 

Test set:
0    15067
1     4833
Name: future_flare, dtype: int64 



In [7]:
# extract X and y from training dataframe
X_train = df_train.drop(['future_flare'], axis=1).to_numpy()
y_train = df_train['future_flare'].to_numpy()

# extract X and y from test dataframe
X_test = df_test.drop(['future_flare'], axis=1).to_numpy()
y_test = df_test['future_flare'].to_numpy()

print('Train:', X_train.shape, y_train.shape)
print('Test: ', X_test.shape, y_test.shape)

Train: (39800, 20) (39800,)
Test:  (19900, 20) (19900,)


Let's construct now the model

In [8]:
# define model
model = Sequential()
model.add(Bidirectional(LSTM(20, activation='relu'), input_shape=(X_train.shape[1], 1)))
model.add(Dense(30, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[f1_m, 'accuracy'])

# Calculate the weights for each class so that we can balance the data
print(model.summary())

2023-03-07 15:21:09.828521: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-03-07 15:21:09.829747: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-03-07 15:21:09.829921: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (83dc2cdd3c94): /proc/driver/nvidia/version does not exist
2023-03-07 15:21:09.834536: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional (Bidirectiona  (None, 40)               3520      
 l)                                                              
                                                                 
 dense (Dense)               (None, 30)                1230      
                                                                 
 dense_1 (Dense)             (None, 10)                310       
                                                                 
 dense_2 (Dense)             (None, 1)                 11        
                                                                 
Total params: 5,071
Trainable params: 5,071
Non-trainable params: 0
_________________________________________________________________
None


In [9]:
# fit model
model.fit(X_train, y_train, epochs=20, verbose=1)

Epoch 1/20
Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089
1244/1244 [==============================] - 43s 25ms/step - loss: 0.2583 - f1_m: 0.7822 - accuracy: 0.9090
Epoch 2/20
1244/1244 [==============================] - 29s 23ms/step - loss: 0.1884 - f1_m: 0.8366 - accuracy: 0.9219
Epoch 3/20
1244/1244 [==============================] - 33s 27ms/step - loss: 0.1789 - f1_m: 0.8438 - accuracy: 0.9265
Epoch 4/20
1244/1244 [==============================] - 29s 23ms/step - loss: 0.1751 - f1_m: 0.8451 - accuracy: 0.9267
Epoch 5/20
1244/1244 [==============================] - 33s 26ms/step - loss: 0.1704 - f1_m: 0.8490 - accuracy: 0.9287
Epoch 6/20
1244/1244 [==============================] - 33s 26ms/step - loss: 0.1666 - f1_m: 0.8461 - accuracy: 0.9290
Epoch 7/20
1244/1244 [==============================] - 27s 22ms/step - loss: 0.1625 

Training results

In [11]:
scores = model.evaluate(X_train, y_train, verbose=0)
y_pred = np.round(model.predict(X_train), 0)

print("### Evaluation on test set ###")
print("Accuracy: %.2f" % (accuracy_score(y_pred, y_train)))
print("F1 score: %.2f" % (f1_score(y_pred, y_train)))
#Create confusion matrix and normalizes it over predicted (columns)
result = confusion_matrix(y_train, y_pred)
print(result)


1244/1244 [==============================] - 20s 16ms/step
### Evaluation on test set ###
Accuracy: 0.94
F1 score: 0.88
[[29097   622]
 [ 1641  8440]]


Test results

In [10]:
scores = model.evaluate(X_test, y_test, verbose=0)
y_pred = np.round(model.predict(X_test), 0)

print("### Evaluation on test set ###")
print("Accuracy: %.2f" % (accuracy_score(y_pred, y_test)))
print("F1 score: %.2f" % (f1_score(y_pred, y_test)))
#Create confusion matrix and normalizes it over predicted (columns)
result = confusion_matrix(y_test, y_pred)
print(result)


622/622 [==============================] - 10s 15ms/step
### Evaluation on test set ###
Accuracy: 0.94
F1 score: 0.87
[[14702   365]
 [  833  4000]]


# Interpolation

In [14]:
theta_train_list = [0.01, 3]
theta_test_list  = [0.1, 0.5]

df_train, df_test = get_dataset_split(grid_X, grid_y, idx, window_size=20, overlap_size=18)

2it [00:00, 9157.87it/s]
2it [00:00, 5143.23it/s]


In [15]:
# extract X and y from training dataframe
X_train = df_train.drop(['future_flare'], axis=1).to_numpy()
y_train = df_train['future_flare'].to_numpy()

# extract X and y from test dataframe
X_test = df_test.drop(['future_flare'], axis=1).to_numpy()
y_test = df_test['future_flare'].to_numpy()

print('Train:', X_train.shape, y_train.shape)
print('Test: ', X_test.shape, y_test.shape)

Train: (19800, 20) (19800,)
Test:  (9900, 20) (9900,)


Same model as before

In [17]:
# define model
model = Sequential()
model.add(Bidirectional(LSTM(20, activation='relu'), input_shape=(X_train.shape[1], 1)))
model.add(Dense(30, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[f1_m, 'accuracy'])
# fit model
model.fit(X_train, y_train, epochs=20, verbose=1)

Epoch 1/20
619/619 [==============================] - 30s 37ms/step - loss: 0.2952 - f1_m: 0.7891 - accuracy: 0.8998
Epoch 2/20
619/619 [==============================] - 24s 39ms/step - loss: 0.2077 - f1_m: 0.8108 - accuracy: 0.9127
Epoch 3/20
619/619 [==============================] - 22s 35ms/step - loss: 0.2022 - f1_m: 0.8140 - accuracy: 0.9142
Epoch 4/20
619/619 [==============================] - 19s 31ms/step - loss: 0.1955 - f1_m: 0.8182 - accuracy: 0.9160
Epoch 5/20
619/619 [==============================] - 22s 35ms/step - loss: 0.1926 - f1_m: 0.8235 - accuracy: 0.9181
Epoch 6/20
619/619 [==============================] - 15s 24ms/step - loss: 0.1875 - f1_m: 0.8287 - accuracy: 0.9192
Epoch 7/20
619/619 [==============================] - 14s 23ms/step - loss: 0.1859 - f1_m: 0.8278 - accuracy: 0.9192
Epoch 8/20
619/619 [==============================] - 14s 23ms/step - loss: 0.1834 - f1_m: 0.8351 - accuracy: 0.9214
Epoch 9/20
619/619 [==============================] - 14s 23ms/s

Training results

In [18]:
scores = model.evaluate(X_train, y_train, verbose=0)
y_pred = np.round(model.predict(X_train), 0)

print("### Evaluation on test set ###")
print("Accuracy: %.2f" % (accuracy_score(y_pred, y_train)))
print("F1 score: %.2f" % (f1_score(y_pred, y_train)))
#Create confusion matrix and normalizes it over predicted (columns)
result = confusion_matrix(y_train, y_pred)
print(result)

619/619 [==============================] - 7s 11ms/step
### Evaluation on test set ###
Accuracy: 0.94
F1 score: 0.88
[[14166   550]
 [  691  4393]]


Test results

In [19]:
scores = model.evaluate(X_test, y_test, verbose=0)
y_pred = np.round(model.predict(X_test), 0)

print("### Evaluation on test set ###")
print("Accuracy: %.2f" % (accuracy_score(y_pred, y_test)))
print("F1 score: %.2f" % (f1_score(y_pred, y_test)))
#Create confusion matrix and normalizes it over predicted (columns)
result = confusion_matrix(y_test, y_pred)
print(result)

310/310 [==============================] - 3s 10ms/step
### Evaluation on test set ###
Accuracy: 0.93
F1 score: 0.85
[[7115  407]
 [ 302 2076]]


Results are still approximately similar to the standard case

# Extrapolation

In [20]:
theta_train_list = [0.1, 0.5]
theta_test_list  = [0.01, 3]

df_train, df_test = get_dataset_split(grid_X, grid_y, idx, window_size=20, overlap_size=18)

2it [00:00, 2299.51it/s]
2it [00:00, 10118.95it/s]


In [21]:
# extract X and y from training dataframe
X_train = df_train.drop(['future_flare'], axis=1).to_numpy()
y_train = df_train['future_flare'].to_numpy()

# extract X and y from test dataframe
X_test = df_test.drop(['future_flare'], axis=1).to_numpy()
y_test = df_test['future_flare'].to_numpy()

print('Train:', X_train.shape, y_train.shape)
print('Test: ', X_test.shape, y_test.shape)

Train: (19800, 20) (19800,)
Test:  (9900, 20) (9900,)


Same model as before

In [22]:
# define model
model = Sequential()
model.add(Bidirectional(LSTM(20, activation='relu'), input_shape=(X_train.shape[1], 1)))
model.add(Dense(30, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[f1_m, 'accuracy'])
# fit model
model.fit(X_train, y_train, epochs=20, verbose=1)

Epoch 1/20
619/619 [==============================] - 22s 26ms/step - loss: 0.2536 - f1_m: 0.7645 - accuracy: 0.9054
Epoch 2/20
619/619 [==============================] - 12s 19ms/step - loss: 0.1739 - f1_m: 0.8495 - accuracy: 0.9319
Epoch 3/20
619/619 [==============================] - 11s 18ms/step - loss: 0.1567 - f1_m: 0.8603 - accuracy: 0.9379
Epoch 4/20
619/619 [==============================] - 11s 18ms/step - loss: 0.1485 - f1_m: 0.8703 - accuracy: 0.9402
Epoch 5/20
619/619 [==============================] - 14s 22ms/step - loss: 0.1429 - f1_m: 0.8787 - accuracy: 0.9441
Epoch 6/20
619/619 [==============================] - 13s 21ms/step - loss: 0.1406 - f1_m: 0.8782 - accuracy: 0.9434
Epoch 7/20
619/619 [==============================] - 11s 19ms/step - loss: 0.1337 - f1_m: 0.8797 - accuracy: 0.9457
Epoch 8/20
619/619 [==============================] - 12s 19ms/step - loss: 0.1324 - f1_m: 0.8814 - accuracy: 0.9458
Epoch 9/20
619/619 [==============================] - 11s 19ms/s

Training results

In [23]:
scores = model.evaluate(X_train, y_train, verbose=0)
y_pred = np.round(model.predict(X_train), 0)

print("### Evaluation on test set ###")
print("Accuracy: %.2f" % (accuracy_score(y_pred, y_train)))
print("F1 score: %.2f" % (f1_score(y_pred, y_train)))
#Create confusion matrix and normalizes it over predicted (columns)
result = confusion_matrix(y_train, y_pred)
print(result)

619/619 [==============================] - 5s 6ms/step
### Evaluation on test set ###
Accuracy: 0.95
F1 score: 0.90
[[14447   370]
 [  572  4411]]


Test results

In [24]:
scores = model.evaluate(X_test, y_test, verbose=0)
y_pred = np.round(model.predict(X_test), 0)

print("### Evaluation on test set ###")
print("Accuracy: %.2f" % (accuracy_score(y_pred, y_test)))
print("F1 score: %.2f" % (f1_score(y_pred, y_test)))
#Create confusion matrix and normalizes it over predicted (columns)
result = confusion_matrix(y_test, y_pred)
print(result)

310/310 [==============================] - 2s 8ms/step
### Evaluation on test set ###
Accuracy: 0.91
F1 score: 0.81
[[7146  305]
 [ 562 1887]]
